In [1]:
"""Librairies nécessaires"""
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
"""Changement de directory pour lire les fichiers"""
import os
os.getcwd() #trouver le directory actuel 
os.chdir('/home/onyxia/work/Projet-python-2A')

Le but du nettoyage est le suivant :

-se restreindre aux ventes d'appartement, donc retirer les ventes de maisons et de locaux commerciaux (en prenant garde aux ventes groupées)

-ajouter aux appartements le nombre de leurs dépendances :
    ici, il faut un traitement différencié selon les fichiers dvf:
        -en 2022 et 2023 le numérot de lot permet de retrouver facilement, pour chaque appartement, ses dépendances s'il en a.
        -avant 2022, le numéro de lot ne permet pas ce traitement : on évacue les ventes groupées contenant plus d'un
        appartement, et on associe à chaque appartement (isolé) les éventuelles dépendances qui sont dans la même vente.

-faire le prorata du prix de vente selon la surface réelle batie, dans les cas de ventes contenant plus d'un appartement (pour les années 2022 et 2023)

-ajouter le prix au m^2 selon la surface réelle batie

-ajouter quand c'est possible la surface carrez (et le prix au m^2 carrez)

-ne garder que les appartements pour lesquels on a une surface, une localisation, un prix

In [3]:
"""
On définit une fonction qui consiste à ne conserver que les ventes d'appartement,
en prenant garde aux ventes groupées qui contiennent des appartments et des maisons ou des locaux industriels :
on retire donc toutes les ventes (groupées) qui contiennent ce type de bien.
"""

def retirer_maison_locaux(df):
    
    #on regroupe les identifiants de vente en comptant le nombre de maisons et de locaux
    ventes_group = df.groupby('id_mutation').agg({'maison_ind': 'sum', 'local_ind': 'sum'})
    
    #on localise les identifiants des ventes où il y a une maison ou un local
    ventes_a_supp = ventes_group.loc[(ventes_group['maison_ind']>0) | (ventes_group['local_ind']>0)] 

    #on convertit en dataframe
    ventes_a_supp = pd.DataFrame(ventes_a_supp) 

    #on créée la liste des ventes à supprimer en récupérant l'index
    list_ventes_a_supp = ventes_a_supp.index.tolist() 

    #on retire les ventes 
    df = df.set_index('id_mutation') #on passe l'id_de vente en index
    df = df.drop(list_ventes_a_supp) #on drop tous les id présents dans la liste à supprimer

    #on repasse l'identifiant de mutation en colonne et on reset l'index
    df = df.reset_index()

    return df

In [4]:
"""
On définit la fonction qui va ajouter le nombre de dépendances pour les fichiers dvf d'avant 2022
"""

def ajouter_dependances_avant_2022(df):
    #df : dataframe

    #on regroupe selon l'identifiant de mutation, en sommant les indicatrices du type dépendance et appartement
    ventes_group = df.groupby('id_mutation').agg(
        nombre_dependances=('dependance_ind','sum'),
        nombre_appartement=('appartement_ind','sum')
    )

    #on ne garde que les ventes où il y a un unique appartement
    ventes_group = ventes_group.loc[ventes_group['nombre_appartement']==1]

    #on conserve seulement la colonne 'nombre_dependances'
    ventes_group = ventes_group.drop('nombre_appartement', axis=1)

    #on passe l'id_de vente en index dans le dataframe initial
    df = df.set_index('id_mutation') 

    #on effectue un merge selon l'index
    df = pd.merge(df, ventes_group, left_index=True, right_index=True) 
        
    #on repasse l'identifiant de mutation en colonne et on reset l'index
    df = df.reset_index()

    return df

In [5]:
"""
On définit la fonction de nettoyage d'une vente (année 2022 et 2023), qui consiste à :
- ajouter à chaque appartement son nombre de dépendances
- faire le prorata du prix selon la surface réelle, dans le cas où il y a plusieurs appartements dans la vente
"""

def nettoyage_vente_2022_2023(id, df):
    #id : indice de la vente
    #df : dataframe
    
    mini_df = df.loc[(df['id_mutation']==id)] #on récupère la vente
    mini_df_appart = df.loc[(df['id_mutation']==id) & (df['appartement_ind']==1)] #on récupère les appartements de cette vente
    index_app = mini_df_appart.index.tolist() #index des appartements
    
    nb_tot_app = mini_df_appart['appartement_ind'].sum() #nombre total d'appartments dans la vente
    
    if nb_tot_app > 1: #s'il y a au moins un appartement dans la vente, on essaie de récupérer pur chaque appart. la surface du lot associé
        
        mini_df_group_lot_dep = mini_df.groupby('lot1_numero').sum()['dependance_ind'] #pour chaque numéro de lot, on récupère le nombre de dépendance

        #pour chaque appartement dans la vente, on met à jour les informations du nombre de dépendance
        for index in index_app:
            num_lot = mini_df.loc[index,'lot1_numero'] #on récupère le numéro de lot associé à l'appartement
            nb_dépendance = mini_df_group_lot_dep[num_lot] #on récupère le nombre de dépendances
            df.loc[index,'nombre_dependances']=nb_dépendance #on met à jour le dataframe

        #enfin, on met à jour le prix
        
        if nb_tot_app > 1: #s'il y a plus d'un appartement dans la vente, on met à jour le prix au prorata de la surface réelle
            surface_totale = mini_df_appart['surface_reelle_bati'].sum() #surface réelle totale
            prix_lot = mini_df_appart.loc[index_app[0]]['valeur_fonciere'] #on récupère la valeur foncière totale
            
            for index in index_app: #pour chaque appartement, on fait le prorata selon sa surface réelle
                surface_app = mini_df_appart.loc[index]['surface_reelle_bati'] #on récupère la surface de l'appartement
                df.loc[index, 'prix'] = prix_lot*(surface_app)/(surface_totale) #on met à jour le prix
                
        else: #si un seul appartement, rien à faire
            pass
            
    elif nb_tot_app == 1: #s'il y a un seul appartement dans la vente, on lui ajoute toutes les dépendances de la vente
        
        nb_dep = mini_df['dependance_ind'].sum()
        
        df.loc[index_app[0],'nombre_dependances']=nb_dep #on met à jour le dataframe
            
    else: #si aucun appartement dans la vente, rien à faire
        pass

In [6]:
"""Fonction qui nettoie les fichiers dvf d'avant 2022"""

def nettoyer_avant_2022(emplacement_données):

    df = pd.read_csv(emplacement_données, sep=",", low_memory=False) #lecture du fichier

    #On sélectionne les variables d'intérêt
    variables_interet = ['id_mutation', 'date_mutation', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'ancien_id_parcelle', 'lot1_numero', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales', 'longitude', 'latitude']

    #variables où sont stockées les surfaces en m2 carrez
    list_lots_carrez = ['lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez', 
                 'lot5_surface_carrez']

    df = df.loc[:,variables_interet] #on ne retient que les variables qui nous intéressent
    df = df.loc[df['nature_mutation']=='Vente'] #on ne garde que les ventes

    #On crée les variables indicatrices du type de logement, utiles pour le traitement
    df['appartement_ind']= [1.0 if type_local == 'Appartement' else 0.0 for type_local in df.type_local]
    df['dependance_ind'] = df['type_local'].apply(lambda x: 1.0 if x == 'Dépendance' else 0.0)    
    df['maison_ind']= [1.0 if type_local == 'Maison' else 0.0 for type_local in df.type_local]
    df['local_ind']= [1.0 if type_local == 'Local industriel. commercial ou assimilé' else 0.0 for type_local in df.type_local]
    
    #On initialise les variables surface_carrez et prix 
    df['surface_carrez']=df[list_lots_carrez].max(axis=1) #on prend la valeur max renseignée dans les colonnes carrez
    df['prix']=df['valeur_fonciere'].round() #par défaut égal à la valeur foncière
    
    #On retire ensuite les ventes contenant des maisons et des locaux, grâce à la fonction retirer_maison_locaux
    df = retirer_maison_locaux(df)

    #Puis on met à jour le nombre de dépendances
    df = ajouter_dependances_avant_2022(df)

    #Il ne reste plus qu'à conserver seulement les lignes de ventes d'appartement
    df = df.loc[df['appartement_ind']==1.0]

    #On créée les variables du prix au m2
    df['prix_au_m2_carrez']= (df['prix']/df['surface_carrez']).round() #on met à jour le prix au m2 carrez
    df['prix_au_m2_reel']= (df['prix']/df['surface_reelle_bati']).round() #on met à jour le prix au m2 réel
    
    #Variables finales qu'on veut conserver    
    var_fin = ['id_mutation', 'date_mutation', 'prix', 'type_local',
       'nombre_dependances', 'surface_reelle_bati', 'surface_carrez',
       'nombre_pieces_principales', 'prix_au_m2_carrez', 'prix_au_m2_reel', 'adresse_numero', 'adresse_nom_voie',
       'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune',
       'code_departement', 'id_parcelle', 'longitude', 'latitude']
    
    df = df.loc[:,var_fin] #on ne garde que les variables finales
    df = df[var_fin] #on réorganise l'ordre des variables

    #On ne garde que les lignes pour lesquelles on a une localisation, un prix, une surface réelle 
    df = df.loc[(df['longitude']>0) & (df['latitude']>0) & (df['prix']>0) & (df['surface_reelle_bati']>0)]

    #On exporte le fichier    
    fichier_sortie = f"1.1) Données clean/{emplacement[17:len(emplacement)-4]}_clean.csv"
    df.to_csv(fichier_sortie)

In [7]:
"""Fonction qui nettoie les fichiers dvf de 2022 et 2023"""

def nettoyage_2022_2023(emplacement_données):

    df = pd.read_csv(emplacement_données, sep=",", low_memory=False) #lecture du fichier

    #On sélectionne les variables d'intérêt
    variables_interet = ['id_mutation', 'date_mutation', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'ancien_id_parcelle', 'lot1_numero', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales', 'longitude', 'latitude']

    #variables où sont stockées les surfaces en m2 carrez
    list_lots_carrez = ['lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez', 
                 'lot5_surface_carrez']

    df = df.loc[:,variables_interet] #on ne retient que les variables qui nous intéressent
    df = df.loc[df['nature_mutation']=='Vente'] #on ne garde que les ventes

    #On crée les variables indicatrices du type de bien, pour faciliter le traitement
    df['appartement_ind']= [1.0 if type_local == 'Appartement' else 0.0 for type_local in df.type_local]
    df['dependance_ind'] = df['type_local'].apply(lambda x: 1.0 if x == 'Dépendance' else 0.0)    
    df['maison_ind']= [1.0 if type_local == 'Maison' else 0.0 for type_local in df.type_local]
    df['local_ind']= [1.0 if type_local == 'Local industriel. commercial ou assimilé' else 0.0 for type_local in df.type_local]

    #On initialise les autres variables dont on a besoin
    df['surface_carrez']=df[list_lots_carrez].max(axis=1) #on prend la valeur max renseignée dans les colonnes carrez
    df['prix']=df['valeur_fonciere'].round() #par défaut égal à la valeur foncière
    df['nombre_dependances']=0.0

    # Par la suite on aura besoin de la variable 'lot1_numero' : on lui donne un format approprié, et on retire les lignes qui ne la précisent pas
    # ces dernières sont peu nombreuses, et en général elles ne précisent pas non plus le type de bien immobilier, donc on peut les retirer
    
    df['lot1_numero'] = pd.to_numeric(df['lot1_numero'], errors='coerce')
    df = df.loc[df['lot1_numero']>0]
    
    # On applique la fonction retirer_maison_locaux afin de ne garder que les ventes d'appartement
    df = retirer_maison_locaux(df)

    tqdm.pandas()

    def nettoyage_vente_wrapper(id):
        # Utilisation de tqdm.pandas() pour suivre la progression
        return nettoyage_vente_2022_2023(id, df)

    df['id_mutation'].progress_apply(nettoyage_vente_wrapper)
    
    #les variables finales qu'on veut garder
    var_fin = ['id_mutation', 'date_mutation', 'prix', 'type_local',
       'nombre_dependances', 'surface_reelle_bati', 'surface_carrez',
       'nombre_pieces_principales', 'prix_au_m2_carrez', 'prix_au_m2_reel', 'adresse_numero', 'adresse_nom_voie',
       'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune',
       'code_departement', 'id_parcelle', 'longitude', 'latitude']

    #On ne garde que les lignes correspondant aux appartements
    df = df.loc[df['appartement_ind']==1.0]

    #On met à jour les variables de prix au m2 (carrez et réel)
    df['prix_au_m2_carrez']= (df['prix']/df['surface_carrez']).round(0) 
    df['prix_au_m2_reel']= (df['prix']/df['surface_reelle_bati']).round(0)

    #On arrondit le prix
    df['prix']=df['prix'].round(0)
    
    #On ne garde que les variables finales et on réorganise l'ordre
    df = df.loc[:,var_fin]
    df = df[var_fin]
    
    #on ne garde que les lignes pour lesquelles on a un prix, une surface, une localisation
    df = df.loc[(df['surface_reelle_bati']>0) & (df['longitude']>0) & (df['prix']>0) & (df['latitude']>0)]

    #on exporte les données en csv
    fichier_sortie = f"1.1) Données clean/{emplacement[17:len(emplacement)-4]}_clean.csv"
    df.to_csv(fichier_sortie)

In [8]:
"""On nettoie les fichiers des années 2018 à 2021"""

for année in ["2018","2019","2020","2021"]:
    emplacement = f"0) Données brutes/DVF_75_{année}.csv"
    nettoyer_avant_2022(emplacement)

In [25]:
"""On nettoie les fichiers des années 2022 et 2023"""

"""ATTENTION : 
temps d'exécution très long
"""

for année in ["2022","2023"]:
    emplacement = f"0) Données brutes/DVF_75_{année}.csv"
    nettoyage_2022_2023(emplacement)